In [6]:
import os
import pandas as pd
import pyarrow.parquet as pq
import shutil

def transferir_dados_conditions_para_curated(diretorio_raw, diretorio_curated, chunksize=1000):
    # Cria o diretÃ³rio de curated caso nÃ£o exista
    os.makedirs(diretorio_curated, exist_ok=True)

    # Lista de arquivos Parquet no diretÃ³rio raw
    arquivos_raw = [f for f in os.listdir(diretorio_raw) if f.endswith('.parquet')]

    if not arquivos_raw:
        print("ðŸš¨ Nenhum arquivo Parquet encontrado no diretÃ³rio raw.")
        return

    # Itera sobre os arquivos Parquet no diretÃ³rio raw
    for arquivo in arquivos_raw:
        caminho_arquivo_raw = os.path.join(diretorio_raw, arquivo)
        
        # LÃª o arquivo Parquet com pyarrow
        parquet_file = pq.ParquetFile(caminho_arquivo_raw)

        # Define o caminho de saÃ­da no diretÃ³rio curated
        caminho_saida = os.path.join(diretorio_curated, arquivo)

        # VariÃ¡vel para controlar se o arquivo jÃ¡ foi criado
        arquivo_criado = False

        # Itera sobre os pedaÃ§os (chunks) do arquivo Parquet
        for i in range(0, parquet_file.num_row_groups, chunksize):
            # Leitura do lote de dados
            df_chunk = parquet_file.read_row_group(i).to_pandas()

            # âœ… Filtra os registros de Condition no chunk
            df_condition = df_chunk[df_chunk['resource_resourceType'] == 'Condition']

            if not df_condition.empty:
                # Se o arquivo ainda nÃ£o foi criado, cria o arquivo Parquet inicial
                if not arquivo_criado:
                    df_condition.to_parquet(caminho_saida, engine='pyarrow', index=False)
                    print(f"âœ… Arquivo de conditions salvo em: {caminho_saida}")
                    arquivo_criado = True
                else:
                    # Caso o arquivo jÃ¡ tenha sido criado, append dos dados
                    df_condition.to_parquet(caminho_saida, engine='pyarrow', index=False, compression='snappy', append=True)
                    print(f"âœ… Lote de conditions adicionado em: {caminho_saida}")

        # Opcional: Mover o arquivo original para outro diretÃ³rio (se necessÃ¡rio)
        # shutil.move(caminho_arquivo_raw, 'diretorio_backup')  # Caso queira mover o arquivo original para backup

# ðŸ”¥ Exemplo de uso
diretorio_raw = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw'    # Substitua pelo caminho real
diretorio_curated = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition'    # Substitua pelo caminho real
chunksize = 1000  # Tamanho do chunk (lote)

# Chama a funÃ§Ã£o para transferir os dados
transferir_dados_conditions_para_curated(diretorio_raw, diretorio_curated, chunksize)

âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_001.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_002.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_003.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_004.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_005.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_006.parquet
âœ… Arquivo de conditions salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_007.parquet
âœ… Arquivo de condi